In [ ]:
%run /edfi_fetch_urls

In [ ]:
# workspace = "dev"
# apiVersion = "5.2"
# moduleName = "Ed-Fi10"
# schoolYear = None
# districtId = None

In [ ]:
districtPath = districtId if districtId != None else "All"
schoolYearPath = schoolYear if schoolYear != None else "All"

parameterized = False

In [ ]:
%run /edfi_py

In [ ]:
if parameterized == True:
    edfiEntitiesPath = f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/etl_entities/current_run_data'
    
    edfiEntities, _ = edfi.listSpecifiedEntities(edfiEntitiesPath)
    entitiesToFetch = edfiEntities
else:
    entitiesToFetch = 'All'

In [ ]:
exception = None
try:
    if len(entitiesToFetch) != 0:
        edfiClient = EdFiClient(workspace, 
                                kvName, 
                                moduleName, 
                                authUrl, 
                                dataManagementUrl, 
                                changeQueriesUrl, 
                                dependenciesUrl, 
                                apiVersion, 
                                batchLimit, 
                                minChangeVer, 
                                maxChangeVer, 
                                schoolYear, 
                                districtId)
        edfiClient.landEntities(entitiesToFetch)
    else:
        logger.info("No entities to be fetched has been specified")
except Exception as e:
    exception = e

In [ ]:
if exception != None:
    raise exception